# **CRÉDITOS**

# 1. Distribución de créditos por género

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "queretaro" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_queretaro',
 'df_financiamientos_2020_queretaro',
 'df_financiamientos_2021_queretaro',
 'df_financiamientos_2022_queretaro',
 'df_financiamientos_2023_queretaro',
 'df_financiamientos_2024_queretaro']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'id_jul_23':'id','m2_contruido': 'm2_construido','m_construido': 'm2_construido','Metros_construido':'m2_construido','segmento':'categoria','Category':'categoria','m_total':'m2_total','m_construido':'m2_construido','Antiguedad':'antiguedad'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','sexo']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

## Conteo de valores únicos de sexo

In [4]:
conteo_total = {}
# Iterar sobre la lista de DataFrames
for df in dataframes_list:
    # Calcular el conteo de valores únicos de 'sexo' en el DataFrame actual
    conteo_actual = df['sexo'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total
    for sexo, conteo in conteo_actual.items():
        conteo_total[sexo] = conteo_total.get(sexo, 0) + conteo

# Convertir el diccionario a un DataFrame
conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['sexo', 'conteo_total'])
conteo_total_df = conteo_total_df[conteo_total_df['sexo'] != 0]

conteo_total_df

,sexo,conteo_total
0,1,27478
1,2,22704


In [5]:
conteo_total_df['sexo'] = conteo_total_df['sexo'].replace({
    1: 'Hombre', 2:'Mujer'
})
conteo_total_df['sexo'].unique()

array(['Hombre', 'Mujer'], dtype=object)

In [6]:
# Calcular el total de registros
sex = conteo_total_df['sexo']
conteo_totall = conteo_total_df['conteo_total']
total_registros = conteo_totall.sum()
print("Total de registros: ", total_registros)

# Calcular el porcentaje que representa cada número de baños único del total
porcentajes = (conteo_totall / total_registros) * 100

# Crear el gráfico de pastel
colores_personalizados = ['#9500ff','#FF0059'] 
fig = go.Figure(data=[go.Pie(
    labels=sex, 
    values=porcentajes, 
    sort=False, marker=dict(colors=colores_personalizados,))])
fig.update_layout(
    #title="Porcentaje de créditos"
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)


# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo,  carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_pie_creditos_genero',  carpeta='assets/graficas')

fig.show()

Total de registros:  50182


In [8]:
# Calcular el total de registros
sex = conteo_total_df['sexo']
conteo_totall = conteo_total_df['conteo_total']
total_registros = conteo_totall.sum()
print("Total de registros: ", total_registros)

# Calcular el porcentaje que representa cada número de baños único del total
porcentajes = (conteo_totall / total_registros) * 100

# Crear el gráfico de pastel
colores_personalizados = ['#9500ff', '#FF0059'] 
fig = go.Figure(data=[go.Pie(
    labels=sex, 
    values=porcentajes, 
    sort=False,
    marker=dict(colors=colores_personalizados),
    textinfo='label+percent',  # Muestra la etiqueta y el porcentaje
    textposition='inside'      # Coloca las etiquetas fuera del gráfico
)])

fig.update_layout(
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (izquierda, derecha, arriba, abajo)
    showlegend=False  # Oculta la leyenda
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Exportar
guardar_grafico_como_html(fig, 'g_pie_creditos_genero', carpeta='assets/graficas')

fig.show()


Total de registros:  50182


# 2. Destino del crédito

In [7]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['modalidad']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [8]:
# Diccionario para conteo total de valores únicos
conteo_total = {}

# Iterar sobre la lista de DataFrames
for df in dataframes_list:
    df = df.copy() # Usando copia de df
    df['modalidad'] = df['modalidad'].replace({
            1: 'Nueva',
            2: 'Mejoramientos',
            3: 'Usada',
            4: 'Otros programas'
        })
    # Calcular el conteo de valores únicos de 'modalidad' en el DataFrame actual
    conteo_actual = df['modalidad'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total
    for modalidad, conteo in conteo_actual.items():
        conteo_total[modalidad] = conteo_total.get(modalidad, 0) + conteo

# Convertir el diccionario a un DataFrame
conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['modalidad', 'conteo_total'])
# Imprimir el DataFrame con el conteo total
conteo_total_df

,modalidad,conteo_total
0,Nueva,26666
1,Usada,13183
2,Mejoramientos,5582
3,Otros programas,4949


In [9]:
# Calcular el total de registros
total_registros = conteo_total_df['conteo_total'].sum()
# Calcular el porcentaje que representa cada modalidad del total
conteo_total_df['porcentaje'] = (conteo_total_df['conteo_total'] / total_registros) * 100
print("Total de registros: ", total_registros)

# Crear un diccionario con el orden deseado
orden_modalidades = {
    1: 'Nueva',
    2: 'Mejoramientos',
    3: 'Usada',
    4: 'Otros programas'
}
# Reindexar el DataFrame según el orden deseado
conteo_total_df = conteo_total_df.set_index('modalidad').reindex(orden_modalidades.values()).reset_index()
conteo_total_df

Total de registros:  50380


,modalidad,conteo_total,porcentaje
0,Nueva,26666,52.929734
1,Mejoramientos,5582,11.079794
2,Usada,13183,26.167130
3,Otros programas,4949,9.823343


In [10]:
fig = go.Figure()
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#0fffdb','#F79CB9']
porcentajes = conteo_total_df['porcentaje']
# Agregar los datos como barras
fig.add_trace(go.Bar(
    x=conteo_total_df['modalidad'],  # Eje x: Modalidades
    y=conteo_total_df['porcentaje'],  # Eje y: Porcentajes
    marker_color=colores,  # Especifica los colores de las barras
    text=porcentajes,  # Texto que se mostrará en las barras (porcentaje)
    textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
    texttemplate='%{text:.1f}%',  # Formato del texto (porcentaje con un decimal)
))

# Personalizar el diseño de la gráfica
fig.update_layout(
    #title='Créditos por la antigüedad de la vivienda',  # Título de la gráfica
    xaxis=dict(title='Modalidad'),  # Título del eje x
    yaxis=dict(
        title='Créditos por antigüedad', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
         gridwidth=1,
    ),  
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_bar_creditos_destino',carpeta='assets/graficas')

fig.show()